<a href="https://colab.research.google.com/github/Komal77rao/Data-Eng-Modules/blob/main/index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recipes API Lab

### Introduction

In this lesson, we'll practice working with Python scripts while using a recipes API.

### Getting Set Up

Begin by setting up a folder to place our code called `src`.  Then, inside of that folder, create a file called `client.py`.  Outside of the `src` folder, create the `console.py` and begin by importing all of the code in the `client` module.

### Accessing the API

For this lesson, we'll use the API from [The MealDB](https://www.themealdb.com/).  Here is a link to [the documentation](https://www.themealdb.com/api.php).

We also have tests in the `tests` folder.  From the `recipes-api` folder, run Pytest to view the tests.  Write functions for the following:

* `recipe_url` - Take a look at how to `Search a meal by name` in the documentation.  This method allows us to pass in the name of a recipe, and returns the appropriate url.  

* `main_ingredient_url` - Look at how to filter by main ingredient in the documentation.  Write a function that takes in a main ingredient, and returns the appropriate url.

* `request_by_recipe` - Takes in a meal name, and then makes a request to the api to return the recipe for the related meal.  (Remember to use the console to check your work).  And of course, run the tests.

* `request_by_main_ingredient` - Takes in an ingredient name, and then makes a request to the api to return the a list of related recipes.

* `id_url` - Takes in an id, and returns the url to search by id.



In [ ]:
import requests

def request_by_recipe(meal_name):
  url = f'https://www.themealdb.com/api/json/v1/1/search.php?s={meal_name}'
  response = requests.get(url)
  recipe = response.json()
  return recipe


request_by_recipe('Pizza')['meals'][0]

{'idMeal': '53014',
 'strMeal': 'Pizza Express Margherita',
 'strDrinkAlternate': None,
 'strCategory': 'Miscellaneous',
 'strArea': 'Italian',
 'strInstructions': '1 Preheat the oven to 230°C.\r\n\r\n2 Add the sugar and crumble the fresh yeast into warm water.\r\n\r\n3 Allow the mixture to stand for 10 – 15 minutes in a warm place (we find a windowsill on a sunny day works best) until froth develops on the surface.\r\n\r\n4 Sift the flour and salt into a large mixing bowl, make a well in the middle and pour in the yeast mixture and olive oil.\r\n\r\n5 Lightly flour your hands, and slowly mix the ingredients together until they bind.\r\n\r\n6 Generously dust your surface with flour.\r\n\r\n7 Throw down the dough and begin kneading for 10 minutes until smooth, silky and soft.\r\n\r\n8 Place in a lightly oiled, non-stick baking tray (we use a round one, but any shape will do!)\r\n\r\n9 Spread the passata on top making sure you go to the edge.\r\n\r\n10 Evenly place the mozzarella (or oth

In [ ]:

def main_ingredient_url(main_ingredient):
    url = f'https://www.themealdb.com/api/json/v1/1/filter.php?i={main_ingredient}'
    return url


def request_by_main_ingredient(main_ingredient):
  url = main_ingredient_url(main_ingredient)
  response = requests.get(url)
  recipe = response.json()
  return recipe

request_by_main_ingredient('Pizza')

{'meals': None}

In [ ]:
def id_url(id):
  url = f'https://www.themealdb.com/api/json/v1/1/lookup.php?i={id}'
  response = requests.get(url)
  id = response.json()
  return id['meals'][0]['idMeal']

### Coercing the Data

Now let's create an `adapter.py` file inside of the `src` directory.  This will be in charge of coercing the data.  Be sure to import this in the console.

* `meal_recipe` - Write a function that takes the name of a meal and then hits the api to returns just the text of the corresponding recipe.  This should use the `request_by_recipe` function.

* `meal_names_and_ids` - Write a function that takes a list of sample meals and returns just the name and id of each meal.  Look at the tests to see how this works.  You can copy the `sample_meals` variable into the console to work with this.

In [ ]:
def meal_recipe(meal_name):
  recipe_name = request_by_recipe(meal_name)
  return recipe_name['meals'][0]['strInstructions']

def meala_names_and_ids(sample_meals):
  return [{'name': meal['strMeal'], 'id': meal['idMeal']} for meal in sample_meals]

sample_meals = [{'strMeal': 'Brown Stew Chicken', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/sypxpx1515365095.jpg', 'idMeal': '52940'}, {'strMeal': 'Chicken & mushroom Hotpot', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/uuuspp1511297945.jpg', 'idMeal': '52846'}, {'strMeal': 'Chicken Alfredo Primavera', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/syqypv1486981727.jpg', 'idMeal': '52796'}, {'strMeal': 'Chicken Basquaise', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/wruvqv1511880994.jpg', 'idMeal': '52934'}, {'strMeal': 'Chicken Congee', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/1529446352.jpg', 'idMeal': '52956'}, {'strMeal': 'Chicken Handi', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/wyxwsp1486979827.jpg', 'idMeal': '52795'}, {'strMeal': 'Kentucky Fried Chicken', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/xqusqy1487348868.jpg', 'idMeal': '52813'}, {'strMeal': 'Kung Pao Chicken', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/1525872624.jpg', 'idMeal': '52945'}, {'strMeal': 'Pad See Ew', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/uuuspp1468263334.jpg', 'idMeal': '52774'}, {'strMeal': 'Piri-piri chicken and slaw', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/hglsbl1614346998.jpg', 'idMeal': '53039'}, {'strMeal': 'Thai Green Curry', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/sstssx1487349585.jpg', 'idMeal': '52814'}]
meala_names_and_ids(sample_meals)


[{'name': 'Brown Stew Chicken', 'id': '52940'},
 {'name': 'Chicken & mushroom Hotpot', 'id': '52846'},
 {'name': 'Chicken Alfredo Primavera', 'id': '52796'},
 {'name': 'Chicken Basquaise', 'id': '52934'},
 {'name': 'Chicken Congee', 'id': '52956'},
 {'name': 'Chicken Handi', 'id': '52795'},
 {'name': 'Kentucky Fried Chicken', 'id': '52813'},
 {'name': 'Kung Pao Chicken', 'id': '52945'},
 {'name': 'Pad See Ew', 'id': '52774'},
 {'name': 'Piri-piri chicken and slaw', 'id': '53039'},
 {'name': 'Thai Green Curry', 'id': '52814'}]

### Adding Tests

Next take a look at the `test_adapter_again.py` file.  Fill in the tests for two new functions: `meal_names` and `meal_ids`.  These two functions should work similarly to the `meal_names_and_ids`.  Write each test and then get the test to pass.

* `meal_names` - For the `meal_names` function, test that the given the sample_meals it returns just a list of strings with each string being the meal name

* `meal_ids` - For the `meal_ids` function, test that the given the `sample_meals` it returns just a list of strings with each string being the meal id.

In [ ]:
def meal_names(sample_meals):
  return [meal['strMeal'] for meal in sample_meals]


def test_meal_names():
  assert meal_names(sample_meals) ==   ['Brown Stew Chicken',
 'Chicken & mushroom Hotpot',
 'Chicken Alfredo Primavera',
 'Chicken Basquaise',
 'Chicken Congee',
 'Chicken Handi',
 'Kentucky Fried Chicken',
 'Kung Pao Chicken',
 'Pad See Ew',
 'Piri-piri chicken and slaw',
 'Thai Green Curry']

test_meal_names()

In [ ]:
def meal_ids(sample_meals):
  return [meal['idMeal'] for meal in sample_meals]


def test_meal_ids():
  assert meal_ids(sample_meals) ==   ['52940',
 '52846',
 '52796',
 '52934',
 '52956',
 '52795',
 '52813',
 '52945',
 '52774',
 '53039',
 '52814']

meal_ids(sample_meals)

['52940',
 '52846',
 '52796',
 '52934',
 '52956',
 '52795',
 '52813',
 '52945',
 '52774',
 '53039',
 '52814']

### Back to the test_adapter

* `meal_ingredients` - If we look at the response from the api, we'll see that each recipe can return up to twenty ingredients.  Write a function that takes in a single meal and returns a list of ingredients.  View the related test for details.
> **Hint**: You can see in the test file, we placed a `sample_meal` at the top of the file.  It may make sense to place this in the `console.py` file so that you can interact with a sample meal.

> **Other hint**: Begin by seeing if use to Python to create a list of `strIngredients` from 1 to 20, and then use this list to select the appropriate ingredients.
```python
['strIngredient1', 'strIngredient2', 'strIngredient3', 'strIngredient4', 'strIngredient5', 'strIngredient6', 'strIngredient7', 'strIngredient8', 'strIngredient9', 'strIngredient10', 'strIngredient11', 'strIngredient12', 'strIngredient13', 'strIngredient14', 'strIngredient15', 'strIngredient16', 'strIngredient17', 'strIngredient18', 'strIngredient19', 'strIngredient20']
```

In [1]:

sample_meal = {'idMeal': '52772', 'strMeal': 'Teriyaki Chicken Casserole', 'strDrinkAlternate': None,
'strCategory': 'Chicken', 'strArea': 'Japanese',
 'strInstructions': 'Preheat oven to 350° F. Spray a 9x13-inch baking pan with non-stick spray.\r\nCombine soy sauce, ½ cup water, brown sugar, ginger and garlic in a small saucepan and cover. Bring to a boil over medium heat. Remove lid and cook for one minute once boiling.\r\nMeanwhile, stir together the corn starch and 2 tablespoons of water in a separate dish until smooth. Once sauce is boiling, add mixture to the saucepan and stir to combine. Cook until the sauce starts to thicken then remove from heat.\r\nPlace the chicken breasts in the prepared pan. Pour one cup of the sauce over top of chicken. Place chicken in oven and bake 35 minutes or until cooked through. Remove from oven and shred chicken in the dish using two forks.\r\n*Meanwhile, steam or cook the vegetables according to package directions.\r\nAdd the cooked vegetables and rice to the casserole dish with the chicken. Add most of the remaining sauce, reserving a bit to drizzle over the top when serving. Gently toss everything together in the casserole dish until combined. Return to oven and cook 15 minutes. Remove from oven and let stand 5 minutes before serving. Drizzle each serving with remaining sauce. Enjoy!', 'strMealThumb': 'https://www.themealdb.com/images/media/meals/wvpsxx1468256321.jpg', 'strTags': 'Meat,Casserole', 'strYoutube': 'https://www.youtube.com/watch?v=4aZr5hZXP_s',
  'strIngredient1': 'soy sauce', 'strIngredient2': 'water', 'strIngredient3': 'brown sugar', 'strIngredient4': 'ground ginger', 'strIngredient5': 'minced garlic', 'strIngredient6': 'cornstarch', 'strIngredient7': 'chicken breasts', 'strIngredient8': 'stir-fry vegetables', 'strIngredient9': 'brown rice', 'strIngredient10': '', 'strIngredient11': '', 'strIngredient12': '', 'strIngredient13': '', 'strIngredient14': '', 'strIngredient15': '', 'strIngredient16': None, 'strIngredient17': None, 'strIngredient18': None,
  'strIngredient19': None, 'strIngredient20': None, 'strMeasure1': '3/4 cup', 'strMeasure2': '1/2 cup', 'strMeasure3': '1/4 cup', 'strMeasure4': '1/2 teaspoon', 'strMeasure5': '1/2 teaspoon', 'strMeasure6': '4 Tablespoons', 'strMeasure7': '2', 'strMeasure8': '1 (12 oz.)', 'strMeasure9': '3 cups', 'strMeasure10': '', 'strMeasure11': '',
 'strMeasure12': '', 'strMeasure13': '', 'strMeasure14': '', 'strMeasure15': '', 'strMeasure16': None, 'strMeasure17': None, 'strMeasure18': None, 'strMeasure19': None, 'strMeasure20': None, 'strSource': None, 'strImageSource': None, 'strCreativeCommonsConfirmed': None, 'dateModified': None}


def find_ingredients_from(sample_meal):
  ingredients = []
  for k,v in sample_meal.items():
    for number in range(1,21):
      if k == f'strIngredient{number}' and v:    #!= None and v != ""): no need to specify None, 0, "" as they are falsy values
        ingredients.append(v)
  return ingredients


find_ingredients_from(sample_meal)

['soy sauce',
 'water',
 'brown sugar',
 'ground ginger',
 'minced garlic',
 'cornstarch',
 'chicken breasts',
 'stir-fry vegetables',
 'brown rice']

In [2]:

def find_ingredients_from(sample_meal):
  ingredients = []
  for k,v in sample_meal.items():
    if k.startswith('strIngredient') and v:
      ingredients.append(v)
  return ingredients


find_ingredients_from(sample_meal)


['soy sauce',
 'water',
 'brown sugar',
 'ground ginger',
 'minced garlic',
 'cornstarch',
 'chicken breasts',
 'stir-fry vegetables',
 'brown rice']

In [ ]:
#one way to do it if we dont have the previous function

# def find_ingredient_measurements(sample_meal):
#   ingredients = []
#   measures = []

#   for k,v in sample_meal.items():
#     for number in range(1,21):
#       if k == f'strIngredient{number}' and (v != None and v != ""):
#         meal_ingredient = v
#         ingredients.append(meal_ingredient)

#       elif k == f'strMeasure{number}' and (v != None and v != ""):
#         meal_measure = v
#         measures.append(meal_measure)

#   ingredients_measures = dict(zip(ingredients,measures))
#   return ingredients_measures

# find_ingredient_measurements(sample_meal)

#other way is to use the previous function that provides the list of ingredients

def find_ingredient_measurements(sample_meal):
  ingredients = find_ingredients_from(sample_meal)
  measures = []

  for k,v in sample_meal.items():
    for number in range(1,21):
      if k == f'strMeasure{number}' and v:   #(v != None and v != ""):
        measures.append(v)

  ingredients_measures = dict(zip(ingredients,measures))
  return ingredients_measures

find_ingredient_measurements(sample_meal)


{'soy sauce': '3/4 cup',
 'water': '1/2 cup',
 'brown sugar': '1/4 cup',
 'ground ginger': '1/2 teaspoon',
 'minced garlic': '1/2 teaspoon',
 'cornstarch': '4 Tablespoons',
 'chicken breasts': '2',
 'stir-fry vegetables': '1 (12 oz.)',
 'brown rice': '3 cups'}

* `ingredients_and_measurements` - Write a function that takes a single meal and returns a dictionary where the keys are the ingredients and the value is each corresponding measuremment needed.

Ok that's it.  Review how to work with different files and practice by adding to this codebase and your tests.